### This script is taking all the acknowledgement text, processing it, and then calling paraphrasing model to perform text based Augmentation. Then save the result as a image(.png) format. Finally, it goes through all the images and perform image transformation and save the result as .png format.

In [1]:
## importing the modules from ETDAugmentation script
from ipynb.fs.full.ETDaugmentation import *
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
import numpy as np
import cv2
import glob
import re

In [2]:
class ACK_TEXT_IMG_AUG:
    
    def list_of_phrases(self, phrases_ack):
        phrases_list = []
        self.phrases_ack = phrases_ack
        for text in self.phrases_ack:
            ack = parser.preprocess_ack(text)
            ack_paraphrase = augmentation.paraphrased_text(ack)
            ack_list = [ack_paraphrase]
            phrases_list.append(ack_list)
        
        return phrases_list

    ## This function is taking all the augmented text and wrapping it so that it can fit into the page
    ## it also adds heading to the top of the page

    def list_of_ack(self, ack_text):
        list_ack = []
        self.ack_text = ack_text
        for text in self.ack_text:
            for row in text:
                ack_text_wrap = augmentation.wrap_text(row)
                ack_title = "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tACKNOWLEDGEMENTS\n" + ack_text_wrap
                list_ack.append(ack_title)
        return list_ack
    
    def text_on_img(self, fileName, text, size):
        self.fileName = fileName
        self.text = text
        self.size = size
        self.W = 2360
        self.H = 3200
        font = ImageFont.truetype('NimbusMonoPS-Bold.otf', size)
        image = Image.new(mode = "RGB", size = (self.W, self.H), color = "white")
        draw = ImageDraw.Draw(image)
        w, h = draw.textsize(text)
        ## Adjust the texual position in a page and draw the text on a image
        '''
        dedication -- 10 width and height 6; 
        ack -- 15 width and 6 height; 
        gabs -- 13 width and height 16;
        abstract -- 20 width and height 6
        '''
        draw.text(((self.W-w)/15,(self.H-h)/6), text, font=font, fill=(0,0,0), spacing=60)
        image.save(fileName)
        
        return image
    
    ## This function is calling text_on_image function and save the image to the directory
    def ack_aug(self, ack_text_title):
        self.ack_text_title = ack_text_title
        for n, row in enumerate(self.ack_text_title):
            fileName = ("/home/mchou001/Label-Acknowledgement/aug_text{}.png".format(n))
            save_image = self.text_on_img(fileName, row, 40) ## dedication -- 43, ack and abstract - 40, gabs -- 38
        
        return save_image
    
    def img_transform(self, img_files):
        self.img_files = img_files
        
        img_augmentation = iaa.Sequential([
            iaa.Affine(rotate=(-6, 6)),
            iaa.AdditiveGaussianNoise(scale=(10, 50)),
            iaa.SaltAndPepper(p=0.1),
            iaa.GaussianBlur(sigma=0.5),
            iaa.LinearContrast(alpha=1),
            iaa.PerspectiveTransform(scale=0.025, keep_size=True)
        ], random_order = True)
        
        lower = 0
        upper = 1000
        while lower < upper:
            for n, images in enumerate(self.img_files[0:]):
                aug_image = img_augmentation(image=cv2.imread(images))
                cv2.imwrite('./Label-Acknowledgement/501/aug{}.png'.format(lower),aug_image)
                lower += 1

In [3]:
if __name__ == "__main__":
    
    parser = Preprocessor()
    augmentation = Augmentation()
    
    acknowledgement_text = ACK_TEXT_IMG_AUG()
    
    etd_data = pd.read_csv('ETD_aug.csv')
    etd_data.set_index("class", inplace = True)

    label_ack = etd_data.loc["Label-Acknowledgement"]
    phrases_ack = label_ack['text']
    
    ack = acknowledgement_text.list_of_phrases(phrases_ack)
    
    ack_ = acknowledgement_text.list_of_ack(ack)
    
    ack_text_aug = acknowledgement_text.ack_aug(ack_)
    
    img_path = sorted(glob.glob('./Label-Acknowledgement/*.png'), key = parser.numericalSort)
    
    augmented_img = acknowledgement_text.img_transform(img_path)
    
    %%time 
    [ x**2 for x in range(10000)]

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 22.4 µs
